In [ ]:
#일정량 이상의 마이너스 값이 들어가면 그거 학습 안시킬 것임.

k fold? 만큼 학습시킬것.
그다음? 카드 클릭하고 화면 클릭하는 경우만 가능하게 하고 그렇게 안했을 경우 train dataset에 넣지 말것.

In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import collections
import random
import matplotlib.pyplot as plt
import pyautogui as pag
import PIL
import torchvision.transforms as transforms
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import time
import seaborn as sns
from torch.distributions import Categorical

pag.FAILSAFE = False


import argparse
import torch

import torch.multiprocessing as _mp
from torch.distributions import Categorical
import torch.nn.functional as F


import numpy as np
import shutil
from torch.utils.tensorboard import SummaryWriter


from random import sample

from pynput import keyboard
from pynput import mouse
import threading
import tkinter as tk
from tkinter import messagebox

In [2]:
import warnings

warnings.filterwarnings(action='ignore') 

In [3]:
#env.py
class ENV():
    def __init__(self):
        # screenshot의 위치 지정, 클래스 생성할때 가져오기

        # winflg와 lose flag 존재해야 함. 0으로 하는 건 grayscale
        self.winFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/weWin.png', 0)
        self.loseFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemyWin.png', 0)
        # nocard flag
        self.nocardFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/nocard.png')
        # noElixir flag
        self.noelixirFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/noElixir.png')
        self.startGameFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleStart.png')
        self.finishFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleFinish.png')
        self.enemy1Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy1.png')
        self.enemy2Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy2.png')
        self.enemy3Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy3.png')
        self.stop_flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/stop_game_full.png')
        self.main_flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/main_check.png')
        
    def return_state(self, img):
        # 스크린 샷을 인자로 받아와서 모델에 넣을 수 있도록 tensor로 변환
        tf = transforms.ToTensor()
        img_t = tf(img) # time.sleep(0.3)
        img_t = img_t.unsqueeze(0)
        # img_t = img_t.permute(1, 0, 2, 3)

        return img_t

    def check_finish(self,img):
        finishMessage = cv2.cvtColor(self.finishFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(finishMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            return 1

        else:
            return 0

    def check_win(self, img):
        # 게임이 이겼는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag1 = np.array(img.crop((225, 335, 280, 365)))
        checkFlag1 = cv2.cvtColor(checkFlag1, cv2.COLOR_BGR2GRAY)
        win_check = cv2.matchTemplate(checkFlag1, self.winFlag, cv2.TM_CCOEFF_NORMED)
        if win_check > 0.8:
            return 1

        else:
            return 0
           
    def check_lose(self, img):
        # 게임이 졌는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag2 = np.array(img.crop((225, 85, 280, 115)))
        checkFlag2 = cv2.cvtColor(checkFlag2, cv2.COLOR_BGR2GRAY)
        lose_check = cv2.matchTemplate(checkFlag2, self.loseFlag, cv2.TM_CCOEFF_NORMED)
        if lose_check > 0.8:
            return 1

        else:
            return 0

    def check_card(self, img):
        # 카드를 선택하지 않았는지 확인, screenshot을 가져와서 init에 지정된 nocard 이미지와 비교하여 reward 부여
        nocard = cv2.cvtColor(self.nocardFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(nocard, img, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def check_elixir(self, img):
        noElixir = cv2.cvtColor(self.noelixirFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(noElixir, img, cv2.TM_CCOEFF_NORMED)

        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def checkET1(self, img):
        score1 = 0
        checkFlag1 = np.array(img.crop((105, 135, 155, 136)))

        for i in range(50):
            if (checkFlag1[0][i][0] >= 96):
                score1 += 1

        score1 = score1 * 2
        return score1

    def checkET2(self, img):
        score2 = 0
        checkFlag2 = np.array(img.crop((371, 135, 421, 136)))

        for i in range(50):
            if (checkFlag2[0][i][0] >= 96):
                score2 += 1

        score2 = score2 * 2
        return score2

    def checkGameStart(self, img):
        startMessage = cv2.cvtColor(self.startGameFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(startMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0
        
    def checkMainPage(self, img):
        main_check = cv2.cvtColor(self.main_flag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(main_check, img, cv2.TM_CCOEFF_NORMED)
        #win_lose_flagprint(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0
        
    def stop_game(self, img):
        stopMessage = cv2.cvtColor(self.stop_flag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(stopMessage, img, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def enemy1(self, img):
        crownFlag1 = cv2.cvtColor(self.enemy1Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag1, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy2(self, img):
        crownFlag2 = cv2.cvtColor(self.enemy2Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag2, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy3(self, img):
        crownFlag3 = cv2.cvtColor(self.enemy3Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag3, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0

    # def checkOT1
    # def checkOT1
    # def checkOT1

    # 우리 타워와 상대 타워의 hp를 확인하여 reward 부여

    def retryGame(self):
        time.sleep(8)
        pag.click((2860, 875))
        #print("step1 click")
        time.sleep(3)
        
        while(1):
            img = pag.screenshot(region=(2605, 100, 510, 900))
            #print("Asdf")
            if(self.checkMainPage(img)):
                break
            #time.sleep(5)
            
        global train_mode
        mode_running = True    
        def on_key_press(key):
            global result, mode_running
            try:
                if key.char == 'q':#컴퓨터 모드
                    train_mode = 0
                elif key.char == 'p': #사람 모드
                    train_mode = 1
#                     else:
#                         result = 0  # 다른 키 누를 시 0으로 설정

                elif key.char == 'x':
                    pass

                # 'p' 키 또는 'q' 키를 누를 때 리스너 중지
                mode_running = False
            except AttributeError:
                pass
        
        print("select the mode in 5 second")
        # 키보드 리스너 생성
        keyboard_listener = keyboard.Listener(on_press=on_key_press)
        keyboard_listener.start()

       

        time.sleep(5)
        
        
                
        # 키보드 리스너 중지
        keyboard_listener.stop()
        
        print("my_variable =", train_mode)

        pag.click((3070, 185))
        time.sleep(3)
        pag.click((2920, 385))
        time.sleep(3)
        pag.click((2950, 615))



In [4]:
learning_rate = 0.0001
gamma = 0.98
lmbda = 0.95
eps_clip = 0.1
K_epoch = 1
T_horizon = 1
delay_time = 0.3

In [5]:
#model.py
class PPO(nn.Module):
    def __init__(self, num_actions):
        super(PPO, self).__init__()
        
        self.data = []
        
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.actor_linear = nn.Sequential(nn.Linear(64, 64),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(64, num_actions))
        self.critic_linear = nn.Sequential(nn.Linear(64, 64),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(64, 1))
        self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
    
    def pi(self, x, softmax_dim = 0):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # x = self.actor_linear(x.reshape(x.size(0), -1))
        x = self.avgpool(x).reshape(x.size(0), -1)
        x = self.actor_linear(x)
        prob = F.softmax(x, dim = softmax_dim)
        return prob
    
    def v(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.avgpool(x).reshape(x.size(0), -1)
        v = self.critic_linear(x.reshape(x.size(0), -1))
        return v
    
    def put_data(self, transition):
        self.data.append(transition)
        
    def clear_data(self):
        self.data = []
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, done_lst = [], [], [], [], [], []
        
        for transition in self.data:
            s, a, r, s_prime, prob_a, done = transition
             
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append(prob_a)
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
            
#         s_lst1 = np.array(s_lst)
#         s_prime_lst1 = np.array(s_prime_lst)
#         print(s_lst1.shape)
#         print(s_prime_lst1.shape)
        
        
        s_lst = torch.stack(s_lst).cuda()
        s_prime_lst = torch.stack(s_prime_lst).cuda()
        
#         print("s_lst")
#         print(s_lst[0])
#         print("s_prime_lst")
#         print(s_prime_lst[0])
        s_lst = torch.tensor(s_lst[0]).cuda()
        s_prime_lst = torch.tensor(s_prime_lst[0]).cuda()
        #print(f"type_slst = {type(s_lst)}")
        s,a,r,s_prime,done_mask, prob_a = s_lst, torch.tensor(a_lst).cuda(), \
                                          torch.tensor(r_lst).cuda(), s_prime_lst, \
                                          torch.tensor(done_lst, dtype=torch.float).cuda(), torch.tensor(prob_a_lst).cuda()
#         print("111111111")
#         print(s.shape)
#         print("222222222")
#         print(s_prime.shape)
#         s = s.squeeze()
#         s_prime = s_prime.squeeze()
        s, s_prime, = s.cuda(), s_prime.cuda() 
        self.data = []
        return s, a, r, s_prime, done_mask, prob_a
            
    def train_net(self):
        s, a, r, s_prime, done_mask, prob_a = self.make_batch()
        #print(f"s size = {len(s)}, s_prime_size - {len(s_prime)}")
        
        global train_time
        #print(f"a = {a}")
        for i in range(K_epoch):
            train_time += 1
#             print("s_prime")
#             print(s_prime)
#             print(s_prime.shape)
#             print("s")
#             print(s)
#             print(s.shape)
            
            td_target = r + gamma * self.v(s_prime) * done_mask
            delta = td_target - self.v(s)
            delta = delta.to("cpu")
            delta = delta.detach().numpy()           
            advantage_lst = []
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype = torch.float).cuda()
            
            pi = self.pi(s, softmax_dim = 1)
            pi_a = pi.gather(1, a)
            #print(f"pi_a = {pi_a}, prob_A = {prob_a}")
            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            #print(f"advantage = {advantage}")
            #print("=================")
            #print(f"surr1 = {surr1}, surr2 = {surr2}")
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s), td_target.detach())
            #print(f"loss1 = {loss.mean()}")
            writer.add_scalar("Loss/train", loss.mean(), train_time)
            writer.flush()
            self.optimizer.zero_grad()
            loss.mean().backward()
            #loss.backward()
            self.optimizer.step()
            if(train_time%10 == 0):
                print(f"{train_time} train is over")
            
        

In [6]:
#train.py envs = MultipleEnvironments(opt.world, opt.stage, opt.action_type, opt.num_processes)
env = ENV()

In [7]:
#train.py model = PPO(envs.num_states, envs.num_actions)
model = PPO(45).cuda()

In [8]:
PATH= "trained_models/clash_royale_PPO_v2_5500_0913"

In [9]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [10]:
action_list = []
action_list_name = {}
count = 0
for i in range(5):
    for j in range(8):
        action_list.append((2655+j*58.571, 510+i*63.75))
        
        action_list_name[count] = str(i) + ' ' + str(j)
        count+=1
        
action_list.append([2765, 900])
action_list.append([2855, 940])
action_list.append([2950, 900])
action_list.append([3055, 900])
action_list.append([-1, -1])
action_list_name[40] = "card 1"
action_list_name[41] = "card 2"
action_list_name[42] = "card 3"
action_list_name[43] = "card 4"
action_list_name[44] = "rest action"

In [11]:
print(action_list)
print(action_list_name)

[(2655.0, 510.0), (2713.571, 510.0), (2772.142, 510.0), (2830.713, 510.0), (2889.284, 510.0), (2947.855, 510.0), (3006.426, 510.0), (3064.997, 510.0), (2655.0, 573.75), (2713.571, 573.75), (2772.142, 573.75), (2830.713, 573.75), (2889.284, 573.75), (2947.855, 573.75), (3006.426, 573.75), (3064.997, 573.75), (2655.0, 637.5), (2713.571, 637.5), (2772.142, 637.5), (2830.713, 637.5), (2889.284, 637.5), (2947.855, 637.5), (3006.426, 637.5), (3064.997, 637.5), (2655.0, 701.25), (2713.571, 701.25), (2772.142, 701.25), (2830.713, 701.25), (2889.284, 701.25), (2947.855, 701.25), (3006.426, 701.25), (3064.997, 701.25), (2655.0, 765.0), (2713.571, 765.0), (2772.142, 765.0), (2830.713, 765.0), (2889.284, 765.0), (2947.855, 765.0), (3006.426, 765.0), (3064.997, 765.0), [2765, 900], [2855, 940], [2950, 900], [3055, 900], [-1, -1]]
{0: '0 0', 1: '0 1', 2: '0 2', 3: '0 3', 4: '0 4', 5: '0 5', 6: '0 6', 7: '0 7', 8: '1 0', 9: '1 1', 10: '1 2', 11: '1 3', 12: '1 4', 13: '1 5', 14: '1 6', 15: '1 7', 16: 

In [12]:
writer = SummaryWriter()

2023-09-14 17:52:13.951797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
#train.py curr_episode = 0
curr_episode = 0
train_time = 0
play_time = 0
train_mode = 0

In [14]:
x_list = [2645.0, 2698.75, 2752.5, 2806.25, 2860.0, 2913.75, 2967.5, 3021.25, 3075.0]
y_list = [500, 554, 608, 662, 716, 770]


In [15]:

def check_region(x, y):
    x_pos = -1
    y_pos = -1
    area = -1
    if x >= 2725 and x <= 2805 and y >= 855 and y <= 950:
        area = 40
        
    
    elif x >= 2820 and x <= 2900 and y >= 855 and y <= 950:
        area = 41
    
    elif x >= 2915 and x <= 2995 and y >= 855 and y <= 950:
        area = 42
        
    elif x >= 3010 and x <= 3090 and y >= 855 and y <= 950:
        area = 43
    
    else:
        for i in range(len(x_list)-1):
            if x >= x_list[i] and x < x_list[i+1]:
                x_pos = i
                break

        for j in range(len(y_list)-1):
            if y >= y_list[j] and y < y_list[j+1]:
                y_pos = j
                break

        #print(x_pos, y_pos)
#         rboard를 통한 모니터링과 시각화

            

        else:
            area = 8*y_pos + x_pos

    return area


In [16]:
while True:
    curr_episode += 1
    
    
    #reward 초기화
    r = 0
    
    #done 초기화
    done = 0
    
    # enemy tower score / score1 = left, score2 = right
    score1 = 100
    score2 = 100
    
    num_local_steps = 0
    
    #set start_flag
    start_flag = 0
    
    #승리 패배를 기록
    win_lose_flag = 0
    
    #s라는 변수는 이렇다는 것을 보여주기 위한 구문
    img = pag.screenshot(region=(2605, 100, 510, 900))
    s = env.return_state(img).cuda()
    right_click_flag = 0
    play_time += 1
    
    card_click_flag = 0
    real_click_flag = 0
    
    wrong_click = 0
    
    while(1):
    #for _ in range(40):
        #opt 기본 num_local_steps = 512
        #게임 시작 확인 전까지 멈춤 지시
        #print("1111")
        while (start_flag == 0):
            # 스크린샷 찍기
            img = pag.screenshot(region=(2605, 100, 510, 900))
            #time.sleep(0.5)
            #print("2222")
            if (env.checkGameStart(img)):
                print("game start")
                start_flag = 1
                time.sleep(2)
                #현재 이미지 캡처 후 변환
                env.reset().cuda()
                print(" ")
                print("================")
                print(f"train mode = {train_mode}")
                print("================")
                print(" ")
                break
                
#         img = pag.screenshot(region=(2605, 100, 510, 900))
#         s = env.return_state(img).cuda()
        
        for t in range(T_horizon):        
            # 키 이벤트 핸들러 함수
            
                
            
            
            
            
            #num_local_step_count
            num_local_steps += 1
            
            
            
            
            #print("dddd")
            if train_mode == 0:
               

                prob = model.pi(s, softmax_dim=1)
                #print(f"prob = {prob}")
                m = Categorical(prob)
                a = m.sample().item()
                


                if(a >= 40 and a <= 43):
                    right_click_flag = 1

                if(right_click_flag == 1 and a >= 0 and a <= 39):
                    #print("right_click")
                    r+= 50
                    right_click_flag = 0
                    

                #print(f"action = {action_list_name[a]}")
    #             if(a >= 40 and a <= 43):
    #                 card_click_flag = 1 #card가 선택되면 1
    #                 real_click_flag = 1

    #             if(card_click_flag == 0 and a >= 0 and a <= 39):
    #                 #print(f"wrong click")
    #                 a=44

    #             if(card_click_flag == 1 and a >= 0 and a <= 39):
    #                 real_click_flag = 1
    #                 card_click_flag = 0
    #                 print("right put")
    #                 r+=10



                #print("11111")

                if(a == 44):
                    #print("22222")
                    time.sleep(delay_time)


                #print(f"a = {a}")
                #pag 클릭. rest action은 -1 -1 이므로 실행하지 않음
    #             if(a!=44 and real_click_flag == 1):
    #                 # 화면 클릭
    #                 #print("click")
    #                 pag.click(action_list[a][0], action_list[a][1])
    #                 time.sleep(delay_time)
    #                 if card_click_flag == 1 and a >= 0 and a <= 39:
    #                     real_click_flag = 0
                #print("3333")
                if(a!=44):
                    # 화면 클릭
                    #print("asdf11")
                    pag.click(action_list[a][0], action_list[a][1])
                    print("click")
                    time.sleep(0.5)
                    
                    
                #클릭 한번당 reward -1
                r -= 1
                print(f"action = {action_list_name[a]}")
                # 승리 확인
                if (env.check_win(img)):
                    print("win")
                    r += 5000
                    done = 1
                    win_lose_flag = 1

                # 패배 확인
                if (env.check_lose(img)):
                    print("lose")
                    r -= 5000
                    done = 1
                    win_lose_flag = -1

                # no card확인
                if (env.check_card(img)):
                    r -= 20
                    print("no card")
                    wrong_click += 1

                # no elixir확인
                if (env.check_elixir(img)):
                    r -= 100
                    print("no Elixir")
                    wrong_click += 1

                if (env.enemy1(img)):
                    r -= 100
                    print("crown - 1")

                if (env.enemy2(img)):
                    r -= 100
                    print("crown - 2")

                if (env.enemy3(img)):
                    r -= 5000
                    done = 1
                    print("crown - 3")


                # enemy tower reward calculate
                score1_now = env.checkET1(img)
                score2_now = env.checkET2(img)
                #print(f"score1 = {score1}, score1_now = {score1_now}")
                #print(f"score2 = {score2}, score2_now = {score2_now}")
                print(f"left tower HP = {score1}, right tower HP = {score2}")
                print(f"reward = {r}")
                if (score1_now < score1):
                    r += 10 * (score1 - score1_now)
                    score1 = score1_now

                if (score2_now < score2):
                    r += 10 * (score2 - score2_now)
                    score2 = score2_now
                    
                if(r<-10000):
                    print("too bad result")
                    pag.click((2650, 850))
                    #게임 X클릭
                    time.sleep(0.5)
                    img = pag.screenshot(region=(2605, 100, 510, 900))
                    if(env.stop_game(img)):
                        pag.click((2930, 650))
                        model.clear_data()
                        done = 1
                        break
                        
                #현재 이미지 캡처 후 변환
                img = pag.screenshot(region=(2605, 100, 510, 900))
                s_prime = env.return_state(img).cuda()
                s = s_prime
                    
                    
            #print("asdf")
            if train_mode == 1:
                a = -1
                img = pag.screenshot(region=(2605, 100, 510, 900))
                s_prime = env.return_state(img).cuda()
                s = s_prime
                prob = model.pi(s, softmax_dim=1)
                while(1):
                    print("asdfasdfasdfasdfasdf")
                    click_area = 44
                    click_running = True
                    # 마우스 클릭 이벤트 핸들러 함수
                    def on_click(x, y, button, pressed):
                        global click_area, running
                        if pressed and button == mouse.Button.left:
                            x = int(x)  # 왼쪽 클릭된 좌표를 튜플로 설정
                            y = int(y)
                            click_area = check_region(x, y) 
                            running = False # 클릭 이벤트가 발생하면 리스너 중지

                        if pressed and button == mouse.Button.right:
                            if x < 1900:
                                done = 1


                    # 마우스 리스너 생성
                    mouse_listener = mouse.Listener(on_click=on_click)
                    mouse_listener.start()

                    

                    
                    img = pag.screenshot(region=(2605, 100, 510, 900))
                    s_prime = env.return_state(img).cuda()
                    s = s_prime
                    prob = model.pi(s, softmax_dim=1)

                    a = click_area
                    if(a >= 0 and a <= 43):
                        r += 10
                        print(f"action = {action_list_name[a]}")
                        print(f"reward = {r}")
                        mouse_listener.stop()
                        break
                
                
                    

            


            
            
            #print(prob)
            
            
            model.put_data((s, a, r, s_prime, prob[0][a], done))

            
            model.train_net()
            if done:
                break
            
        
        
        
        #print(f"how many time to train? = {train_time}")
            #opt.save_interval = 50
        
        save_interval = 500
        if(train_time % save_interval == 0):
            #print("Episode: {}. Total loss: {}, save_interval = {}".format(curr_episode, total_loss, curr_episode % save_interval))
            torch.save(model.state_dict(),"{}/clash_royale_PPO_v2_{}_0913".format("trained_models", train_time))
        
        if done:
            
            writer.add_scalar("reward/epi", r, play_time)
            writer.flush()
            writer.add_scalar("clicks/epi", num_local_steps, play_time)
            writer.flush()
            writer.add_scalar("win_lose/epi", win_lose_flag, play_time)
            writer.flush()
            writer.add_scalar("wrong_click/epi", wrong_click, play_time)
            writer.flush()
            break
            
    print(f"how many time to choice = {num_local_steps}")        
    print("retry game")
    env.retryGame()

game start
 
train mode = 0
 
click
action = 2 6
left tower HP = 100, right tower HP = 100
reward = -1
click
action = 3 7
left tower HP = 100, right tower HP = 100
reward = -2
click
action = 3 1
left tower HP = 100, right tower HP = 100
reward = -3
click
action = 2 0
left tower HP = 100, right tower HP = 100
reward = -4
click
action = 4 1
no card
left tower HP = 100, right tower HP = 100
reward = -25
click
action = card 1
left tower HP = 100, right tower HP = 100
reward = -26
click
action = 4 0
no card
left tower HP = 100, right tower HP = 100
reward = 3
click
action = 3 4
no card
left tower HP = 100, right tower HP = 100
reward = -18
click
action = 0 5
left tower HP = 100, right tower HP = 100
reward = -19
click
action = 0 6
no card
left tower HP = 100, right tower HP = 100
reward = -40
10 train is over
click
action = 4 0
no card
left tower HP = 100, right tower HP = 100
reward = -61
click
action = 2 4
left tower HP = 100, right tower HP = 100
reward = -62
click
action = card 3
left t

click
action = 2 4
left tower HP = 86, right tower HP = 98
reward = -317
click
action = 1 1
left tower HP = 86, right tower HP = 98
reward = -318
click
action = card 2
no card
left tower HP = 86, right tower HP = 98
reward = -339
click
action = 4 7
left tower HP = 86, right tower HP = 98
reward = -290
click
action = 3 7
left tower HP = 86, right tower HP = 98
reward = -291
click
action = 4 0
crown - 3
left tower HP = 86, right tower HP = 98
reward = -5292
how many time to choice = 62
retry game
select the mode in 5 second
my_variable = 0
game start
 
train mode = 0
 
click
action = 1 2
left tower HP = 100, right tower HP = 100
reward = -1
click
action = 4 4
no card
left tower HP = 100, right tower HP = 100
reward = -22
110 train is over
click
action = 4 0
left tower HP = 100, right tower HP = 100
reward = -23
click
action = 4 1
no card
left tower HP = 100, right tower HP = 100
reward = -44
click
action = 4 4
no card
left tower HP = 100, right tower HP = 100
reward = -65
click
action = 

click
action = card 4
no card
left tower HP = 98, right tower HP = 16
reward = -57
click
action = card 4
left tower HP = 98, right tower HP = 16
reward = -58
click
action = 4 7
left tower HP = 98, right tower HP = 16
reward = -9
click
action = 3 6
left tower HP = 98, right tower HP = 16
reward = -10
click
action = 3 3
left tower HP = 98, right tower HP = 16
reward = -11
click
action = card 3
no card
left tower HP = 98, right tower HP = 16
reward = -32
210 train is over
click
action = 3 7
left tower HP = 98, right tower HP = 16
reward = 17
click
action = 2 2
crown - 2
left tower HP = 98, right tower HP = 16
reward = -84
click
action = 1 3
no card
crown - 2
left tower HP = 98, right tower HP = 16
reward = -205
click
action = 2 6
no card
crown - 2
left tower HP = 98, right tower HP = 16
reward = -326
click
action = 3 0
no card
left tower HP = 98, right tower HP = 16
reward = -347
click
action = card 3
left tower HP = 98, right tower HP = 16
reward = -348
click
action = 3 7
no card
left to


KeyboardInterrupt



In [ ]:
card_click_flag = 1pag.click((2650, 850))
#게임 X

In [ ]:
pag.click((2900, 650))
#게임종료

In [ ]:
img = pag.screenshot('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/main0.png')
#clickEmpty1 = clickEmpty1.crop((2737, 573, 2987, 600))
#print(type(clickEmpty1))
plt.imshow(img)
#clickEmpty2 = np.array(img)
#cv2.imwrite('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/nocard.png', clickEmpty2)
#plt.imshow(clickEmpty1)

In [ ]:
img = pag.screenshot(region=(2605, 100, 510, 900))
#print("Asdf")
if(env.checkMainPage(img)):
    print("As")

In [ ]:
t1 = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/main0.png')
#plt.imshow(t1)
t2 = Image.fromarray(t1)
t3 = t2.crop((3050,165,3105,220))
t4 = np.array(t3)
cv2.imwrite('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/main_check.png', t4)
plt.imshow(t4)

In [ ]:
img = pag.screenshot(region=(2605, 100, 510, 900))
plt.imshow(img)
env.checkMainPage(img)
if(env.checkMainPage(img)):
    print("As")
    
    

In [ ]:
from pynput import keyboard
import time

# 변수 초기화
my_variable = 0

# 키 이벤트 핸들러 함수
def on_key_release(key):
    global my_variable
    try:
        if key.char == 'q':
            my_variable = 1
        elif key.char == 'p':
            my_variable = 2
    except AttributeError:
        pass

# 키보드 리스너 생성
listener = keyboard.Listener(on_release=on_key_release)
listener.start()

# 일정 시간 대기 (예: 10초)
time.sleep(3)

# 키보드 리스너 중지
listener.stop()

# 변수 출력 (이 부분은 필요에 따라 사용)
print("my_variable =", my_variable)

In [ ]:
import pyautogui
import time
for i in range(10):
    x, y = pyautogui.position()
    print(f"x = {x}, y = {y}")
    time.sleep(0.5)

In [ ]:
def myrange(start, end, step):
    r = start
    while(r<end):
        yield r
        r += step

In [ ]:
x_list = [2645.0, 2698.75, 2752.5, 2806.25, 2860.0, 2913.75, 2967.5, 3021.25, 3075.0]
y_list = [500, 554, 608, 662, 716, 770]
click_range = []

In [ ]:
from pynput import mouse
import time

sample_list = []

def check_region(x, y):
    x_pos = -1
    y_pos = -1
    area = -1
    if x >= 2725 and x <= 2805 and y >= 855 and y <= 950:
        area = 40
        
    
    elif x >= 2820 and x <= 2900 and y >= 855 and y <= 950:
        area = 41
    
    elif x >= 2915 and x <= 2995 and y >= 855 and y <= 950:
        area = 42
        
    elif x >= 3010 and x <= 3090 and y >= 855 and y <= 950:
        area = 43
    
    else:
        for i in range(len(x_list)-1):
            if x >= x_list[i] and x < x_list[i+1]:
                x_pos = i
                break

        for j in range(len(y_list)-1):
            if y >= y_list[j] and y < y_list[j+1]:
                y_pos = j
                break

        #print(x_pos, y_pos)
        if x_pos == -1 or y_pos == -1:
            area = -1
            

        else:
            area = 8*y_pos + x_pos

    return area

def on_click(x, y, button, pressed):
    if pressed:
        x = int(x)
        y = int(y)
        temp = check_region(x, y) 
        #print(temp)
        return temp
        
mouse_listener = mouse.Listener(on_click=on_click)
time1 = time.time()
mouse_listener.start()
while(1):
    a = mouse_listener.join()
    if(a == -1):
        print("particular end")
        break
    
    if(time.time() - time1 > 10):
        print('break')
        mouse_listener.stop()
        break
        
mouse_listener.stop()


In [ ]:
x_list

In [ ]:
import threading
from pynput import mouse

# 변수 초기화
result = None
running = True

# 마우스 클릭 이벤트 핸들러 함수
def on_click(x, y, button, pressed):
    global result, running
    if pressed:
        # 클릭된 버튼을 확인하고 변수를 설정
        if button == mouse.Button.left:
            result = 1
        elif button == mouse.Button.right:
            result = 2
        else:
            result = 0  # 다른 버튼 클릭 시 0으로 설정
        
        # 클릭 이벤트가 발생하면 리스너 중지
        running = False

# 마우스 리스너 생성
mouse_listener = mouse.Listener(on_click=on_click)
mouse_listener.start()

# 일정 시간이 지나면 프로그램 종료를 위한 함수
def exit_after_timeout():
    global running
    time.sleep(10)  # 10초 후에 종료
    running = False

# 타임아웃 스레드 시작
timeout_thread = threading.Thread(target=exit_after_timeout)
timeout_thread.start()

# 메인 스레드에서 계속 실행하고, running 변수가 False가 될 때 종료
while running:
    pass

# 마우스 리스너 중지
mouse_listener.stop()

# 결과 출력
print("클릭된 결과:", result)

In [ ]:
env.retryGame()

In [ ]:
def check_region(x, y):
    x_pos = -1
    y_pos = -1
    area = -1
    for i in range(len(x_list)-1):
        if x >= x_list[i] and x < x_list[i+1]:
            x_pos = i
            break
    
    for j in range(len(y_list)-1):
        if y >= y_list[j] and y < y_list[j+1]:
            y_pos = j
            break
    
    print(x_pos, y_pos)
    if x_pos == -1 or y_pos == -1:
        area = -1
        return area
        
    else:
        area = 8*y_pos + x_pos
        
        return area
    
    
print(check_region(3007, 512))

In [ ]:
mouse_listener.stop()

In [ ]:
import tkinter as tk
from tkinter import messagebox

# tkinter 초기화
root = tk.Tk()
root.withdraw()  # 메인 창을 숨깁니다.

# 메시지 박스 표시
messagebox.showinfo("알림", "실행되었습니다.")

# tkinter 종료
root.destroy()

In [ ]:
import threading
from pynput import mouse

# 변수 초기화
result = None
running = True

# 마우스 클릭 이벤트 핸들러 함수
def on_click(x, y, button, pressed):
    global result, running
    if pressed and button == mouse.Button.left:
        result = (x, y)  # 왼쪽 클릭된 좌표를 튜플로 설정
        # 클릭 이벤트가 발생하면 리스너 중지
        running = False
while(1):

    # 마우스 리스너 생성
    mouse_listener = mouse.Listener(on_click=on_click)
    mouse_listener.start()

    # 일정 시간이 지나면 프로그램 종료를 위한 함수
    def exit_after_timeout():
        global running
        time.sleep(2  # 10초 후에 종료
        running = False

    # 타임아웃 스레드 시작
    timeout_thread = threading.Thread(target=exit_after_timeout)
    timeout_thread.start()

    # 메인 스레드에서 계속 실행하고, running 변수가 False가 될 때 종료
    while running:
        pass

    # 마우스 리스너 중지
    mouse_listener.stop()

    # 결과 출력
    print(result)

In [ ]:
import time
from pynput import mouse

# 변수 초기화
last_click_time = 0

# 마우스 클릭 이벤트 핸들러 함수
def on_click(x, y, button, pressed):
    global last_click_time
    if pressed:
        current_time = time.time()
        # 현재 클릭 시간과 마지막 클릭 시간의 차이가 0.3초 이상인 경우에만 처리
        if current_time - last_click_time >= 0.3:
            print(f"클릭 좌표: ({x}, {y})")
        last_click_time = current_time

# 마우스 리스너 생성
mouse_listener = mouse.Listener(on_click=on_click)
mouse_listener.start()

# 메인 스레드에서 계속 실행
try:
    mouse_listener.join()
except KeyboardInterrupt:
    pass
finally:
    mouse_listener.stop()

In [ ]:
2645 500
3075 770


2645 235


1) 2725 855 / 2805 950
2) 2820 855 / 2900 950
3) 2915 855 / 2995 950
4) 3010 855 / 3090 950